In [2]:
import numpy as np
import matplotlib,os
import matplotlib.pyplot as plt
import matplotlib.cm
from sklearn import preprocessing
import pickle
import sys
import pandas as pd
import random
from timeit import default_timer as timer
from scipy import stats
import sklearn
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_predict
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error, r2_score

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

def shuffleRows(arr1):
    arr1=list(arr1)
    random.shuffle(arr1)
    arr1=np.asarray(arr1)
    return arr1

class Scaling:
    def fit(self, train=0, test=0):
        self.scaler = sklearn.preprocessing.StandardScaler()  # z = (x - u) / s
        self.scaler.fit(train)
        self.pcs_train = self.scaler.transform(train) # equivelent with stats.zscore(b, axis=0)
        if type(test)!=int:
            self.pcs_test = self.scaler.transform(test)

class activation_PCA:
    def fit(self, X_train=0, X_test=0, num_components=1000):
        n_samples, n_features = X_train.shape
        self.num_components = min(n_samples, n_features) if num_components > min(n_samples, n_features) else num_components
        self.model = PCA(self.num_components)
        self.model.fit(X_train)
        # print('explained_variance_ratio=',self.model.explained_variance_ratio_)
        # principle components are the coefficients that transform original data into principal vectors space.
        self.pcs_train = self.model.transform(X_train)
        if type(X_test)!=int:
            self.pcs_test = self.model.transform(X_test)

def zscoreForEachRow(data):
    data=data.T
    return ((data-np.mean(data,0))/np.std(data,0)).T